In [45]:
%pip install langchain langchain-openai langchain_chroma langchain_community langchainhub beautifulsoup4


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Zilvinas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [46]:
import getpass
import os
from dotenv import load_dotenv
load_dotenv()

#os.environ["AIKEY"] = getpass.getpass()

token = os.getenv("AIKEY_SECOND")
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4.1-nano"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=model, base_url=endpoint, openai_api_key=token)


In [47]:
import bs4
#from bs4 import BeautifulSoup
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [ ]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2017-06-21-overview/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

embedding = OpenAIEmbeddings(
    base_url="https://models.inference.ai.azure.com",
    model="text-embedding-3-large",
    openai_api_key=openai_api_key
)

vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)

NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

In [54]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

C:\Users\Zilvinas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
